In [3]:
import mapping_noRot as mappNR
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import SimpleITK as sitk
import time
import os

In [5]:
sample_list = ['S130684_L2_left', 'S130684_L2_right',
               'S131318_L1_left', 'S131318_L1_right', 'S131318_L2_left', 'S131318_L2_right',
               'S131318_L3_left', 'S131318_L3_right', 'S131318_L4_left', 'S131318_L4_right',
               'S131318_L5_left', 'S131318_L5_right',
               'S131788_L1_left', 'S131788_L1_right', 'S131788_L2_left', 'S131788_L2_right',
               'S131788_L3_left', 'S131788_L3_right', 'S131788_L4_left', 'S131788_L4_right',
               'S131835_L1_left', 'S131835_L1_right', 'S131835_L2_left', 'S131835_L2_right',
               'S131835_L3_left', 'S131835_L3_right', 'S131835_L4_left', 'S131835_L4_right',
               'S131840_L1_left', 'S131840_L1_right', 'S131840_L2_left', 'S131840_L2_right',
               'S131840_L3_left', 'S131840_L3_right']

In [6]:
sample_code = sample_list[0]
path_project = '/home/biomech/Documents/01_Icotec/'  # General project folder
path_ct = path_project + '01_Experiments/02_Scans/' + sample_code + '/04_Registered/'  # Folder of CT dat
res = 0.0606995

In [15]:
# Area to evaluate
r_mm = 10  # radius in mm
r = int(np.rint(r_mm / res))
length = np.rint(np.array([-50, 0]) / res).astype(int)
print(min(length))
print(max(length))
print(r)
print(825*165**2*np.pi)

-824
0
165
70562134.49503525


In [16]:
i=0
j=0
for z in range(min(length), max(length)):
    for y in range(-r, r):
        for x in range(-r, r):
            if x**2 + y**2 <= r**2:
                i = i+1
            else:
                j = j+1
print(i)
print(j)
print(i/(i+j))

70451176
19282424
0.7851147842056933
